In [1]:
import h5py
import torch
import torch.nn as nn
import random


import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tempfile import TemporaryFile
from scipy.io import loadmat
from scipy.io import savemat
from torch.utils.data import DataLoader, TensorDataset

# Add the Torch_code directory to the Python path
# sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath('../helper'))
import config
import utils
import loader

In [2]:
# Configuration
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

rows = [['3500', '3516']] 
fc = '3p4' #Hz can change to '60'
rowss = "3500_3516"
SNR = np.arange(0, 31, 5) # 0:5:30 dB
outer_file_path = os.path.abspath(os.path.join(config.FILE_PATH, 
                                                '..', 'DeepMIMOv2', 'DeepMIMO_Data', 'Static_BS16', 'freq_symb_1ant_612sub_ver4'))


In [3]:
snr = 0
BATCH_SIZE = 32
rows = [['3500', '3516']] 
H_true = np.empty((0, 2, 612, 14)) # true channel
H_equal = np.empty((0, 2, 612, 14)) # noisy channel # LS channel
H_linear = np.empty((0, 2, 612, 14)) # noisy channel # LS+Linear Interpolated channel
H_practical = np.empty((0, 2, 612, 14)) # noisy channel # Practical Estimated channel

for i in range(len(rows)):
    file_path_partial = 'Gan_' + str(snr) +'_dBOutdoor1_'+ fc +'_1ant_612subcs_Row_' + rows[i][0] +'_' + rows[i][1] + '.mat'

    file_path = os.path.join(outer_file_path, file_path_partial)
    file_path = os.path.normpath(file_path)
    file = h5py.File(file_path, 'r')

    H_true = np.concatenate((H_true, np.array(file['H_data'])), axis = 0) # N_samples x channel(2) x height(614) x width(14)
    H_equal = np.concatenate((H_equal, np.array(file['H_equalized_data'])), axis = 0)
    H_linear = np.concatenate((H_linear, np.array(file['H_linear_data'])), axis=0)
    H_practical = np.concatenate((H_practical, np.array(file['H_practical_data'])), axis=0)

In [8]:
print('min data' , np.min(H_true))
print('max data' , np.max(H_true))
print('mean data ', np.mean(H_true))
print('min abs data ', np.min(np.abs(H_true)))
print('max abs data ', np.max(np.abs(H_true)))

min data -2.763166048680432e-05
max data 2.7671640054904856e-05
mean data  6.890068043088681e-09
min abs data  3.865352482534945e-12
max abs data  2.7671640054904856e-05


# Normalize for training

In [10]:
train_size = np.floor(H_practical.shape[0]*0.9) //BATCH_SIZE *BATCH_SIZE
    # print(train_size)
    # print(train_size/64)
    # print(train_size/input_data.size(0))
train_size = int(train_size)

# -----------------------------------------------------
# 1. When input is H_linear (after LS+LI)
print(f" Training for LS+LI")
# [samples, 2, 612, 14]
# 1.1 Split into training and validation sets for H_NN training
trainData   = H_linear[0:train_size,:,:,:]
trainLabels = H_true[0:train_size,:,:,:]

valData   = H_linear[train_size:,:,:,:]
valLabels = H_true[train_size:,:,:,:]

# Split H_equal, H_linear, H_practical for validation later
H_equal_val = H_equal[train_size:,:,:,:]
H_linear_val = H_linear[train_size:,:,:,:]
H_practical_val = H_practical[train_size:,:,:,:]

 Training for LS+LI


## Approach 1: (min-max scaling) range [0 1]

Validating data also gets normalized by max_min of training data

In [12]:
# training normalization
trainData_min = trainData.min()
trainData_max = trainData.max()
trainLabels_min = trainLabels.min()
trainLabels_max = trainLabels.max()

trainData_normd   = (trainData - trainData_min)/ (trainData_max - trainData_min)
trainLabels_normd = (trainLabels - trainLabels_min)/ (trainLabels_max - trainLabels_min)
valData_normd     = (valData - trainData_min)/ (trainData_max - trainData_min)
valLabels_normd   = (valLabels - trainLabels_min)/ (trainLabels_max - trainLabels_min)

In [13]:
print('min nomrd train ' , np.min(trainLabels_normd))
print('max nomrd train ' , np.max(trainLabels_normd))
print('mean nomrd train ', np.mean(trainLabels_normd))
print('min abs nomrd train ', np.min(np.abs(trainLabels_normd)))
print('max abs nomrd train ', np.max(np.abs(trainLabels_normd)))

min nomrd train  0.0
max nomrd train  1.0
mean nomrd train  0.4997292828098262
min abs nomrd train  0.0
max abs nomrd train  1.0


In [14]:
print('min nomrd validation ' , np.min(valLabels_normd))
print('max nomrd validation ' , np.max(valLabels_normd))
print('mean nomrd validation ', np.mean(valLabels_normd))
print('min abs nomrd validation ', np.min(np.abs(valLabels_normd)))
print('max abs nomrd validation ', np.max(np.abs(valLabels_normd)))

min nomrd validation  0.0023010980613262814
max nomrd validation  0.9989782691760117
mean nomrd validation  0.5000497336187739
min abs nomrd validation  0.0023010980613262814
max abs nomrd validation  0.9989782691760117


## Approach 2: (min-max scaling) range [-1 1]

In [15]:
trainData_normd2   = (trainData - trainData_min)/ (trainData_max - trainData_min) *2 - 1
trainLabels_normd2 = (trainLabels - trainLabels_min)/ (trainLabels_max - trainLabels_min) *2 - 1
valData_normd2     = (valData - trainData_min)/ (trainData_max - trainData_min) *2 - 1
valLabels_normd2   = (valLabels - trainLabels_min)/ (trainLabels_max - trainLabels_min) *2 - 1

In [16]:
print('min nomrd train ' , np.min(trainLabels_normd2))
print('max nomrd train ' , np.max(trainLabels_normd2))
print('mean nomrd train ', np.mean(trainLabels_normd2))
print('min abs nomrd train ', np.min(np.abs(trainLabels_normd2)))
print('max abs nomrd train ', np.max(np.abs(trainLabels_normd2)))

min nomrd train  -1.0
max nomrd train  1.0
mean nomrd train  -0.0005414343803454471
min abs nomrd train  4.316964030692816e-08
max abs nomrd train  1.0


In [17]:
print('min nomrd validation ' , np.min(valLabels_normd2))
print('max nomrd validation ' , np.max(valLabels_normd2))
print('mean nomrd validation ', np.mean(valLabels_normd2))
print('min abs nomrd validation ', np.min(np.abs(valLabels_normd2)))
print('max abs nomrd validation ', np.max(np.abs(valLabels_normd2)))

min nomrd validation  -0.9953978038773474
max nomrd validation  0.9979565383520235
mean nomrd validation  9.946723754790591e-05
min abs nomrd validation  8.901168692121431e-07
max abs nomrd validation  0.9979565383520235


### Check some samples

In [22]:
print('min nomrd train 1 ' , np.min(trainLabels_normd2[1,:,:,:]))
print('max nomrd train 1 ' , np.max(trainLabels_normd2[1,:,:,:]))
print('mean nomrd train 1 ', np.mean(trainLabels_normd2[1,:,:,:]))
print('min abs nomrd train 1 ', np.min(np.abs(trainLabels_normd2[1,:,:,:])))
print('max abs nomrd train 1 ', np.max(np.abs(trainLabels_normd2[1,:,:,:])))

min nomrd validation 1  -0.37007745636651224
max nomrd validation 1  0.3666743730641706
mean nomrd validation 1  0.01144091171114728
min abs nomrd validation 1  0.00017454205069966644
max abs nomrd validation 1  0.37007745636651224


In [23]:
print('min nomrd train 100 ' , np.min(trainLabels_normd2[100,:,:,:]))
print('max nomrd train 100 ' , np.max(trainLabels_normd2[100,:,:,:]))
print('mean nomrd train 100 ', np.mean(trainLabels_normd2[100,:,:,:]))
print('min abs nomrd train 100 ', np.min(np.abs(trainLabels_normd2[100,:,:,:])))
print('max abs nomrd train 100 ', np.max(np.abs(trainLabels_normd2[100,:,:,:])))

min nomrd validation 100  -0.39982342712607843
max nomrd validation 100  0.39510213213291134
mean nomrd validation 100  -0.006273588457452051
min abs nomrd validation 100  0.0004005721202535728
max abs nomrd validation 100  0.39982342712607843


In [24]:
print('min nomrd train 500 ' , np.min(trainLabels_normd2[500,:,:,:]))
print('max nomrd train 500 ' , np.max(trainLabels_normd2[500,:,:,:]))
print('mean nomrd train 500 ', np.mean(trainLabels_normd2[500,:,:,:]))
print('min abs nomrd train 500 ', np.min(np.abs(trainLabels_normd2[500,:,:,:])))
print('max abs nomrd train 500 ', np.max(np.abs(trainLabels_normd2[500,:,:,:])))

min nomrd validation 500  -0.2806238110053211
max nomrd validation 500  0.27837550324699833
mean nomrd validation 500  0.00254815752903729
min abs nomrd validation 500  0.0004748537091898708
max abs nomrd validation 500  0.2806238110053211


## Approach 3: Instance normalization, Range [-1 1] (min-max scaling) 

* Instance norm for H_LI 
* Instance norm for H_true
* Train H_LI_normed to get H_true_normed, then denorm to get H_true, then evaluate

In [52]:
train_min = []
train_max = []
trainLabels_normd3 = np.empty((trainLabels.shape[0], 2, 612, 14))

for i in range(trainLabels.shape[0]):
    sample = trainLabels[i]
    
    # Compute mean and variance for the current sample
    min = sample.min()
    max = sample.max()
    
    trainLabels_normd3[i,:,:,:] = (sample - min) / (max - min) *2 -1

    # Append to lists
    train_min.append(min.item())
    train_max.append(max.item())

In [53]:
print('min nomrd train ' , np.min(trainLabels_normd3))
print('max nomrd train ' , np.max(trainLabels_normd3))
print('mean nomrd train ', np.mean(trainLabels_normd3))
print('min abs nomrd train ', np.min(np.abs(trainLabels_normd3)))
print('max abs nomrd train ', np.max(np.abs(trainLabels_normd3)))

min nomrd train  -1.0
max nomrd train  1.0
mean nomrd train  -0.0005734639920682292
min abs nomrd train  4.4367372198372834e-07
max abs nomrd train  1.0


## Aproach 4: Instance normalization, (standardize)

In [49]:
train_mean = []
train_var = []
trainLabels_normd4 = np.empty((trainLabels.shape[0], 2, 612, 14))

for i in range(trainLabels.shape[0]):
    sample = trainLabels[i]
    
    # Compute mean and variance for the current sample
    mean = sample.mean()
    variance = sample.var()
    
    trainLabels_normd4[i,:,:,:] = (sample - mean) / np.sqrt(variance)
    
    # Append to lists
    train_mean.append(mean.item())
    train_var.append(variance.item())

# # Convert lists to tensors or numpy arrays if needed
# means = torch.tensor(means)
# variances = torch.tensor(variances)


In [51]:
trainLabels_normd4.shape

print('min nomrd train ' , np.min(trainLabels_normd4))
print('max nomrd train ' , np.max(trainLabels_normd4))
print('mean nomrd train ', np.mean(trainLabels_normd4))
print('min abs nomrd train ', np.min(np.abs(trainLabels_normd4)))
print('max abs nomrd train ', np.max(np.abs(trainLabels_normd4)))


min nomrd train  -2.7031298970480138
max nomrd train  2.6884923172900796
mean nomrd train  5.568834140846658e-19
min abs nomrd train  3.720750786702688e-07
max abs nomrd train  2.7031298970480138
